# Cuaderno 32: Algoritmo de generación de columnas
## Otro modelo para el problema de corte de material (Cutting Stock Problem)

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\S}{\mathcal{S}}$


Dados: 
* un conjunto de rollos, cada uno de longitud $L$; y,
* $m$ órdenes de corte, cada una de las cuales consiste en cortar de los rollos disponibles $b_i$ items de longitud $w_i$, con $w_i < L$, $i \in \{1, \ldots, m\}$.

El problema del corte de material (*Cutting Stock Problem*) consiste en determinar la manera de satisfacer todas las órdenes de corte empleando la menor cantidad posible de rollos. Consideraremos a continuación una formulación para este problema distinta a la formulación del Cuaderno 31.

Definimos un *patrón de corte* como una manera posible de cortar uno o más items de la longitud requerida por las órdenes a partir de *un* rollo de longitud $L$. Notar que un patrón de corte puede ser representado como un vector $s$ de $m$ componentes enteras que además satisface $\sum_{i=1}^m w_i s_i \leq L$. Es decir, el conjunto de todos los posibles patrones de corte está dado por:
$$
\S :=  \left\{ s \in \ZZ^m_+ \; : \; \sum_{i=1}^m w_i s_i \leq L \right\}.
$$

Definiendo una variable entera $x_s$ que indique el número de veces que se usa cada patrón de corte $s \in S$ en la solución, el problema de corte de material puede formularse como el siguiente programa lineal entero:
\begin{align*}
\min &\sum_{s \in \S} x_s\\ 
& \mbox{s.r.}\\
&\sum_{s \in \S} s_i x_s  \geq b_i, \quad \forall i \in \{1, \ldots, m\},\\
& x_s \in \ZZ, \quad \forall s \in \S.
\end{align*}

Llamaremos *problema master (PM)* al modelo anterior. Generalmente, este programa lineal entero tiene un número astronómico de variables, pues la cantidad de posibles patrones de corte crece exponencialmente como función de $m$. Problemas de este tipo pueden resolverse empleando una técnica conocida como **generación de columnas**.

Definimos al *problema master reducido (PMR)* como el modelo anterior, pero restringido a utilizar únicamente un subconjunto $\S' \subset \S$ de patrones de corte disponibles. Inicialmente, $\S'$ puede contener, por ejemplo, solamente *patrones homogéneos* que cortan un rollo en la mayor cantidad posible de items de la misma longitud (notar que esta cantidad es igual a $\lfloor \frac{L}{w_i} \rfloor$, para $i \in \{1, \ldots, m\}$). Llamando $s^1, \ldots, s^m$ a estos patrones homogéneos, inicialmente PMR tiene la forma:
\begin{align*}
\min &\sum_{i = 1}^m x_{s^i}\\ 
& \mbox{s.r.}\\
& \lfloor \frac{L}{w_i} \rfloor x_{s^i}  \geq b_i, \quad \forall i \in \{1, \ldots, m\},\\
& x_{s^i} \in \ZZ, \quad \forall i \in \{1, \ldots, m\}.
\end{align*}


El algoritmo de generación de columnas empieza por resolver la relajación lineal de PMR. Luego se resuelve el siguiente problema de costeo (PRICING): Determinar si existe algún patrón en $\S \setminus \S'$ tal que el costo reducido de la variable correspondiente en la relajación lineal de PM, respecto a los precios sombra dados por la solución del problema dual de PMR, sea estrictamente negativo. De ser así, se añade este nuevo patrón como una variable adicional a PMR y se repite el proceso. Caso contrario, la solución óptima de la relajación lineal de PMR es también una solución óptima de la relajación lineal de PM y la fase de generación de columnas termina. Finalmente, el modelo PMR se resuelve como un problema lineal entero. 

El algoritmo de generación de columnas permite encontrar la solución óptima de la relajación lineal de PM, aunque no necesariamente la solución óptima entera. Para resolver PM hasta la optimalidad, sería necesario aplicar una nueva fase de generación de columnas en cada nodo del árbol de branch-and-bound. Este nuevo algoritmo se conoce como *Branch-and-Price* y su uso está limitado por su gran complejidad computacional. 

Para el caso del problema del corte de material, sin embargo, el algoritmo (simple) de generación de columna permite obtener soluciones con brechas de optimalidad muy pequeñas.

Implementaremos a continuación el algoritmo de generación de columnas para resolver PM utilizando la interfaz Python del solver Gurobi.


Definimos primero los datos, al igual que en el Cuaderno 31. Generaremos $m$ órdenes de corte, consistentes en cortar items de longitud entera aleatoria entre 1 y 9. La cantidad de items a cortar de cada tipo estará dada por un número aleatorio entre 10 y 50. La longitud de los rollos será $L := 10$.

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import random as rm
import math

# iniciar generador de numeros aleatorios
rm.seed(0)

# número de órdenes
m = 100

# lista con los índices de las órdenes,
# diccionarios con las longitudes y demandas de cada item
I, w, b = gp.multidict({
    i : (rm.randint(1, 9), rm.randint(10, 50)) 
        for i in range(1, m+1)}
)
    
L = 10

# cantidad maxima de rollos a utilizar
N = sum(int(math.ceil(w[i]*b[i]/(L - w[i]))) for i in I)

# conjuntos de índices de rollos
J = range(1, N+1)

# número de columnas (variables) iniciales en el modelo
K = len(I)

print(w)
print(b)
print(N)

Definimos ahora el modelo reducido inicial PMR, sus variables y su función objetivo. Este modelo contiene únicamente patrones homogéneos que cortan rollos en items de la misma longitud.


In [ ]:
# Crear el objeto modelo
pm = gp.Model('cutting-stock')

# Crear las variables asociadas a patrones homogeneos
x = pm.addVars(I, name="x", vtype=GRB.INTEGER)

# Crear la funcion objetivo
pm.setObjective(x.sum('*'), GRB.MINIMIZE)


Añadimos las restricciones de satisfacción de la demanda. 

In [ ]:
# Restricciones de demanda
r = pm.addConstrs((int(L/w[i])*x[i] >= b[i] for i in I), "r");


Resolvemos ahora la relajación lineal del modelo inicial. La relajación lineal de un modelo de programación lineal entera puede construirse llamando al método `relax` de la clase `Model`.

In [ ]:
# Llamamos primero a update para actualizar los cambios
pm.update()

# Construimos la relajacion lineal del modelo
relax = pm.relax()

# Resolvemos la relacion lineal del modelo
relax.optimize()

### Resolviendo el problema de costeo (PRICING)

El problema de costeo consiste en encontrar, o determinar que no existe, un nuevo patrón de corte $s \in \S$, tal que la variable correspondiente en la relajación lineal de PM tenga costo reducido negativo, con respecto a los valores $\pi_1, \ldots, \pi_m$ de las variables duales de la solución óptima de la relajación lineal de PMR encontrada en el paso anterior.

Suponer que este patrón de corte tiene la forma $s:= (y_1, \ldots, y_m)$, con $y_i$ enteros no negativos tales que $\sum_{i=1}^m w_i y_i \leq L$. El costo reducido de la variable $x_s$ asociada a este patrón es:
$$
\tilde{c}_s := c_s - \sum_{i = 1}^m \pi_i y_i,
$$
donde $c_s = 1$ es el coeficiente de $x_s$ en la función objetivo de PM. Observar que $\tilde{c}_s < 0$ si y solamente si $\sum_{i = 1}^m \pi_i y_i > 1$.

Por lo tanto, el problema de costeo puede reducirse a la solución del siguiente problema de la mochila (*knapsack problem*):
\begin{align*}
\max &\sum_{i = 1}^m \pi_i y_i\\ 
& \mbox{s.r.}\\
&\sum_{i=1}^m w_i y_i \leq L,\\
& y_i \in \ZZ_+, \quad \forall i \in \{1, \ldots, m\}.
\end{align*}

Si la solución óptima de este problema tiene un valor mayor a uno, la misma está asociada a un nuevo patrón de costo reducido negativo. Caso contrario, se demuestra que no existen más patrones con costo reducido negativo y que por tanto la solución óptima de la relajación lineal de PMR es también solución óptima de la relajación lineal de PM.

Para resolver este problema de la mochila, empezamos por recuperar los valores de $\pi_1, \ldots, \pi_m$ asociados a la solución dual del modelo `relax`. Esto se consigue accediendo al atributo `Pi` de cada restricción lineal. A su vez, para acceder a las restricciones lineales del modelo, se emplea el método `getConstrs()` de la clase `Model`.


In [ ]:
# Recuperamos la solucion dual del problema master
pi = {}
i = 1
for c in relax.getConstrs():
    pi[i] = c.Pi
    i+= 1


Construimos ahora el modelo del problema de la mochila, empleando los valores de `pi`, `w` y `L`:

In [ ]:
# Crear un objeto modelo auxiliar para el problema de la mochila
h = gp.Model('knapsack')

# Crear las variables del problema de la mochila
y = h.addVars(I, name="y", vtype=GRB.INTEGER)

# Crear la funcion objetivo
h.setObjective(y.prod(pi, '*'), GRB.MAXIMIZE)

# Agregar la restriccion 
h.addConstr(y.prod(w, '*') <= L, 'capacidad')

h.update()

Resolvemos el problema de la mochila y verificamos si la solución óptima tiene un valor mayor a 1.

In [ ]:
# Resolver modelo de pricing (problema de la mochila)
h.optimize()


Como el valor de la función objetivo es mayor a 1, agregamos una nueva variable al modelo PM, asociada al nuevo patrón encontrado. 

Para ello, construimos en primer lugar un objeto de tipo `Column` que almacenará la columna correspondiente a esta nueva variable. Los coeficientes (no nulos) de la columna se definen llamando al método `addTerms`. Finalmente, se llama a `addVar` para agregar al modelo una nueva variable asociada a esta columna y con coeficiente igual a 1 en la función objetivo.

In [ ]:
# incrementar en 1 el indice maximo de variables
K+= 1

# crear un objeto columna
col = gp.Column()

# definir los coeficientes de la columna usando la solucion del modelo del knapsack
vy = h.getAttr('x', y)
for i in I:
    if vy[i] > 0.1:
        col.addTerms(round(vy[i]), r[i])

# agregar al modelo principal una nueva variable asociada a esta columna
x[K] = pm.addVar(name="x[{}]".format(K), vtype=GRB.INTEGER, obj=1, column= col)

Repetimos el procedimiento de generación de columnas mientras el subproblema de costeo tenga una solución óptima con valor mayor a 1. Para desactivar los mensajes a la pantalla durante la solución de los modelos, fijamos a cero el valor del parámetro `OutputFlag` en ambos modelos

In [ ]:
# desactivar mensajes de salida del modelo pm
pm.Params.OutputFlag = 0
while True:
    # 1. Resolver la relajacion lineal del problema master
    # Llamamos primero a update para actualizar el modelo master
    pm.update()
    # Construimos la relajacion lineal de m
    relax = pm.relax()
    # Resolvemos la relacion lineal de m
    relax.optimize() 

        
    # 2. Construir y resolver el subproblema de costeo (PRICING)
    # Recuperamos la solucion dual del problema master
    pi = {}
    i = 1
    for c in relax.getConstrs():
        pi[i] = c.Pi
        i+= 1
    # Crear un objeto modelo auxiliar para el problema de la mochila
    h = gp.Model('knapsack')
    # desactivar mensajes de salida del modelo pm
    h.Params.OutputFlag = 0
    # Crear las variables del problema de la mochila
    y = h.addVars(I, name="y", vtype=GRB.INTEGER)
    # Crear la funcion objetivo
    h.setObjective(y.prod(pi, '*'), GRB.MAXIMIZE)
    # Agregar la restriccion 
    h.addConstr(y.prod(w, '*') <= L, 'capacidad')
    # Resolver modelo del subproblema
    h.optimize()
    
    # 3. Si el valor optimo de la funcion objetivo del subproblema no supera 1, terminar la generacion de columnas
    if h.ObjVal < 1.1:
        break
        
    # 4. Agregar la nueva columna al problema
    # incrementar en 1 el indice maximo de variables
    K+= 1

    # crear un objeto columna
    col = gp.Column()

    # definir los coeficientes de la columna usando la solucion del modelo del knapsack
    vy = h.getAttr('x', y)
    for i in I:
        if vy[i] > 0.1:
            col.addTerms(round(vy[i]), r[i])

    # agregar al modelo principal una nueva variable asociada a esta columna
    x[K] = pm.addVar(name="x[{}]".format(K), vtype=GRB.INTEGER, obj=1, column= col) 

# la fase de generacion de columnas ha terminado
print("Se generaron {} patrones de corte...".format(K-m))

Finalmente, resolvemos el modelo del problema master como un programa entero. Fijamos un límite para el tiempo de cálculo y una tolerancia para la brecha de optimalidad:

In [ ]:
# Terminar al alcanzar un Gap del 10%
pm.Params.MIPGap = 0.01

# Terminar luego de 240 segundos
pm.Params.TimeLimit = 240

# Activar nuevamente los mensajes de salida
pm.Params.OutputFlag = 1

# Resolver el problema principal
pm.optimize()

Escribir la solución óptima:

In [ ]:
# recuperar variables cuyo valor sea mayor a cero:
for var in pm.getVars():
    if var.getAttr('X') > 0.1:
        # recuperar la columna asociada a cada variable
        col = pm.getCol(var)
        # decodificar el patron correspondiente a col
        L = []
        for i in range(col.size()):
            L.append((int(col.getCoeff(i)), col.getConstr(i).ConstrName))
        S = ['{:2}x{} '.format(k, r) for k,r in L]
        print ('{:2}: [{}]'.format(int(var.getAttr('X')), ''.join(S)))


## Código completo

Se reproduce a continuación el código completo del modelo anterior. Notar que no es necesario generar la primera columna "por separado" como lo hicimos arriba, con el propósito de explicar mejor el algoritmo.

In [ ]:
# Implementación de programas lineales enteros
# Problema del corte de material (Cutting-Stock)
# Implementación con generación de columnas

# Luis M. Torres (EPN 2022)

import gurobipy as gp
from gurobipy import GRB
import random as rm
import math

# iniciar generador de numeros aleatorios
rm.seed(0)

# número de órdenes
m = 100

# lista con los índices de las órdenes,
# diccionarios con las longitudes y demandas de cada item
I, w, b = gp.multidict({
    i : (rm.randint(1, 9), rm.randint(10, 50)) 
        for i in range(1, m+1)}
)
    
L = 10

# cantidad maxima de rollos a utilizar
N = sum(int(math.ceil(w[i]*b[i]/(L - w[i]))) for i in I)

# conjuntos de índices de rollos
J = range(1, N+1)

# número de columnas (variables) iniciales en el modelo
K = len(I)

try:
    # crear el modelo principal
    pm = gp.Model('cutting-stock')

    # desactivar mensajes de salida del modelo pm
    pm.Params.OutputFlag = 0

    # crear las variables asociadas a patrones homogeneos
    x = pm.addVars(I, name="x", vtype=GRB.INTEGER)

    # crear la funcion objetivo
    pm.setObjective(x.sum('*'), GRB.MINIMIZE)

    # restricciones de demanda
    r = pm.addConstrs((int(L/w[i])*x[i] >= b[i] for i in I), "r");

    # Escribir el modelo PM inicial a un archivo
    # m.write('cutting-stock-inicial.lp')
    
    # lazo principal de la generación de columnas
    while True:
        # 1. Resolver la relajación lineal del problema master reducido
        # llamamos primero a update para actualizar el modelo master
        pm.update()
        # construimos la relajacion lineal de pm
        relax = pm.relax()
        # resolvemos la relación lineal de pm
        relax.optimize() 
        
        # 2. Construir y resolver el subproblema de costeo (PRICING)
        # recuperar la solución dual del problema master
        pi = {}
        i = 1
        for c in relax.getConstrs():
            pi[i] = c.Pi
            i+= 1
        # crear un objeto modelo auxiliar para el problema de la mochila
        h = gp.Model('knapsack')
        # desactivar mensajes de salida del modelo h
        h.Params.OutputFlag = 0
        # crear las variables del problema de la mochila
        y = h.addVars(I, name="y", vtype=GRB.INTEGER)
        # crear la función objetivo
        h.setObjective(y.prod(pi, '*'), GRB.MAXIMIZE)
        # agregar la restricción 
        h.addConstr(y.prod(w, '*') <= L, 'capacidad')
        # resolver modelo del subproblema
        h.optimize()
    
        # 3. Si el valor óptimo del subproblema no supera 1, terminar la generación de columnas
        if h.ObjVal < 1.1:
            break
        
        # 4. Agregar la nueva columna al problema
        # incrementar en 1 el indice maximo de variables
        K+= 1
        # crear un objeto columna
        col = gp.Column()
        # definir los coeficientes de la columna usando la solución del modelo del knapsack
        vy = h.getAttr('x', y)
        for i in I:
            if vy[i] > 0.1:
                col.addTerms(round(vy[i]), r[i])
        # agregar al modelo principal una nueva variable asociada a esta columna
        x[K] = pm.addVar(name="x[{}]".format(K), vtype=GRB.INTEGER, obj=1, column= col) 
        
    # la fase de generación de columnas ha terminado
    print("Se generaron {} patrones de corte...".format(K-m))

    # fijar parametros del modelo principal
    # terminar al alcanzar un Gap del 1%
    pm.Params.MIPGap = 0.01
    # terminar luego de 180 segundos
    pm.Params.TimeLimit = 180
    # activar nuevamente los mensajes de salida
    pm.Params.OutputFlag = 1

    # resolver el problema principal
    pm.optimize()
    
    # escribir la solución
    if pm.SolCount > 0 :
        # recuperar variables cuyo valor sea mayor a cero:
        for var in pm.getVars():
            if var.getAttr('X') > 0.1:
                # recuperar la columna asociada a cada variable
                col = pm.getCol(var)
                # decodificar el patron correspondiente a col
                L = []
                for i in range(col.size()):
                    L.append((int(col.getCoeff(i)), col.getConstr(i).ConstrName))
                S = ['{:2}x{} '.format(k, r) for k,r in L]
                print ('{:2}: [{}]'.format(int(var.getAttr('X')), ''.join(S)))
        
except GurobiError as e:
    print('Se produjo un error de Gurobi: codigo: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')